#### このノート
- data: X_mushroom, y_mushroom
- CNF式を再現してみた。
- 1の総数は指定しなかった方法と、した方法
- 単数出力、複数出力



#### 教訓
**変数aに変数bを代入し、その後変数bの値を変える場合、変数aは代入時の値と変わってしまう**

対策
- copy.copy('代入したい値')
- copy.deepcopy('代入したい値')
- '代入したい値'.copy()
- '代入したい値'[:]


**ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()**

対策
- とりあえず`a.all()`を当てはめてみる

https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7163203

In [1]:
import copy
import itertools
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_regression
import random
from pyqubo import Array, OneHotEncInteger, solve_qubo
import scipy.stats

In [2]:
#標準偏差が0である列をなくす
def drop_str0_X(X):
    for each_col in X.columns:
        if X[each_col].std()==0.0:
            X = X.drop(each_col, axis=1)
    return X

In [3]:
X_mushroom = pd.read_csv('../input/X_mushroom.csv', sep=',', index_col=0)
X_mushroom_ad = drop_str0_X(X_mushroom)

num_samples = X_mushroom.shape[0]

In [4]:
X_mushroom_ad.head()

,0,1,2,3,4,5,6,7,8,9,...,109,110,111,112,113,114,115,116,117,118
0,1,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
1,0,1,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,1,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [5]:
y_mushroom = pd.read_csv('../input/y_mushroom.csv', sep=',', index_col=0)['21'] #いずれ21→yとなるはず
y_mushroom.head()

0    0
1    1
2    1
3    0
4    1
Name: 21, dtype: int64

In [6]:
y1_sum = sum(y_mushroom)

In [7]:
type(y_mushroom)

pandas.core.series.Series

In [8]:
def featrure_InnerProduct_sel_mul(X, y, sel_col_num):
    inner_product_list = [np.dot(X[each_col], y) for each_col in X.columns] #内積のリスト
    sort_inner_product_list = sorted(inner_product_list ,reverse=True)
    
    select_cols = []
    for i in range(sel_col_num):
        iindex = inner_product_list.index(sort_inner_product_list[i])
        select_cols.append(X.columns[iindex])
    
    else_cols = list(X.columns)
    for col in select_cols:
        else_cols.remove(col)
    
    return select_cols, else_cols

In [9]:
selected_col_num2 = 2
select_cols, else_cols = featrure_InnerProduct_sel_mul(X_mushroom_ad, y_mushroom, selected_col_num2)

## SATのシステムをオリジナルで書いてみる
- 単数出力、1の総数未指定
- 複数出力、1の総数未指定
- 単数出力、1の総数指定
- 複数出力、1の総数指定

In [10]:
#len, 0と1だけ使用する、という条件だけ指定し、自動でlistを作ってもらう
def make(num_samples):
    shou = num_samples//2
    if num_samples%2 == 0:
        infer_y0 = pd.Series([0, 1]*shou)
    else:
        infer_y0 = pd.Series([0, 1]*shou + [0])
    return infer_y0

In [11]:
infer_y0 = make(num_samples)

In [12]:
dot_list_ori = [np.dot(X_mushroom_ad[i], y_mushroom) - np.dot(X_mushroom_ad[j], y_mushroom) for i in select_cols for j in else_cols]
for each_dot in dot_list_ori:
    if each_dot < 0 :
        print('error is here')
        break
print('ok')

ok


In [13]:
#合っているか確認
def check_decide_y_fuc(X, ori_y, selected_col_num, deci_y):
    select_cols, else_cols = featrure_InnerProduct_sel_mul(X, ori_y, selected_col_num)
    
    dot_list_confirm = [np.dot(X[i], deci_y) - np.dot(X[j], deci_y) for i in select_cols for j in else_cols]
    if len([ll for ll in dot_list_confirm if ll <0]) != 0:
        print('{} is wrong'.format(deci_y.values.tolist()))

In [14]:
check_decide_y_fuc(X_mushroom_ad, y_mushroom, selected_col_num2, y_mushroom)

In [15]:
if len([ll for ll in dot_list_ori if ll <0]) == 0:
    print('there is no problem')

there is no problem


In [16]:
#合っているか確認(複数出力)
def check_decide_y_fuc_mul(X, ori_y, selected_col_num, deci_y_list):
    select_cols, else_cols = featrure_InnerProduct_sel_mul(X, ori_y, selected_col_num)

    for m in deci_y_list:
        dot_list_confirm = [np.dot(X[i], m) - np.dot(X[j], m) for i in select_cols for j in else_cols]
        if len([ll for ll in dot_list_confirm if ll <0]) == 0:
            continue
        else:
            print('{} is wrong'.format(m.values.tolist()))

### 単数出力、1の総数未指定

In [17]:
# 出力一つ
def decide_y(X, ori_y, infer_y, selected_col_num):
    select_cols, else_cols = featrure_InnerProduct_sel_mul(X, ori_y, selected_col_num)
    deci_y = 0
    dot_list = [np.dot(X[i], infer_y) - np.dot(X[j], infer_y) for i in select_cols for j in else_cols]
    if len([ll for ll in dot_list if ll <0]) == 0:
        deci_y = infer_y
        return deci_y
    else:
        for kk in reversed(range(num_samples)):
            for k in reversed(range(kk+1)):
                infer_y[k] = 1 - infer_y[k]
                dot_list = [np.dot(X[i], infer_y) - np.dot(X[j], infer_y) for i in select_cols for j in else_cols]
                if len([ll for ll in dot_list if ll <0]) == 0:
                    deci_y = infer_y
                    return deci_y
                else:
                    infer_y[k] = 1 - infer_y[k]
            infer_y[kk] = 1 - infer_y[kk]
    return deci_y #series

In [ ]:
deci_y0 = decide_y(X_mushroom_ad, y_mushroom, infer_y0, selected_col_num2)
deci_y0

In [ ]:
check_decide_y_fuc(X_mushroom_ad, y_mushroom, selected_col_num2, deci_y0)

In [ ]:
dot_list_confirm0 = [np.dot(X_mushroom_ad[i], deci_y0) - np.dot(X_mushroom_ad[j], deci_y0) for i in select_cols for j in else_cols]
dot_list_confirm0

In [ ]:
# 出力複数
def decide_y_mul(X, ori_y, infer_y, selected_col_num):
    select_cols, else_cols = featrure_InnerProduct_sel_mul(X, ori_y, selected_col_num)
    deci_y_list = []
    dot_list = [np.dot(X[i], infer_y) - np.dot(X[j], infer_y) for i in select_cols for j in else_cols]
    
    if len([ll for ll in dot_list if ll <0]) == 0:
        deci_y_list.append(copy.deepcopy(infer_y))
    for kk in reversed(range(num_samples)):
        for k in reversed(range(kk+1)):
            infer_y[k] = 1 - infer_y[k]
            dot_list = [np.dot(X[i], infer_y) - np.dot(X[j], infer_y) for i in select_cols for j in else_cols]
            if len([ll for ll in dot_list if ll <0]) == 0:
                if len(deci_y_list) == 0:
                    deci_y_list.append(copy.deepcopy(infer_y))
                elif (deci_y_list[-1] == infer_y).all() == False:
                    deci_y_list.append(copy.deepcopy(infer_y))
            infer_y[k] = 1 - infer_y[k]
        infer_y[kk] = 1 - infer_y[kk]
    
    return deci_y_list

In [ ]:
infer_y0 = make(num_samples)
deci_y1 = decide_y_mul(X_mushroom_ad, y_mushroom, infer_y0, selected_col_num2)
deci_y1
#[[1, 1, 0, 1, 0]]

In [ ]:
check_decide_y_fuc_mul(X_mushroom_ad, y_mushroom, selected_col_num2, deci_y1)

In [ ]:
dot_list_confirm1 = [np.dot(X_mushroom_ad[i], deci_y1[0]) - np.dot(X_mushroom_ad[j], deci_y1[0]) for i in select_cols for j in else_cols]
dot_list_confirm1

In [ ]:
# 出力一つ、yに含まれる1を固定
def decide_y_fix_y1(X, ori_y, infer_y, selected_col_num, y1_sum):
    select_cols, else_cols = featrure_InnerProduct_sel_mul(X, ori_y, selected_col_num)
    deci_y = []
    dot_list = [np.dot(X[i], infer_y) - np.dot(X[j], infer_y) for i in select_cols for j in else_cols]
    
    if len([ll for ll in dot_list if ll <0]) == 0 and sum(infer_y) == y1_sum:
        deci_y.append(copy.deepcopy(infer_y))
    else:
        for kk in reversed(range(num_samples)):
            for k in reversed(range(kk+1)):
                infer_y[k] = 1 - infer_y[k]
                dot_list = [np.dot(X[i], infer_y) - np.dot(X[j], infer_y) for i in select_cols for j in else_cols]
                if len([ll for ll in dot_list if ll <0]) == 0 and sum(infer_y) == y1_sum:
                    deci_y.append(copy.deepcopy(infer_y))

                else:
                    infer_y[k] = 1 - infer_y[k]
            infer_y[kk] = 1 - infer_y[kk]
    return deci_y

In [ ]:
infer_y0 = make(num_samples)
deci_y2 = decide_y_fix_y1(X_mushroom_ad, y_mushroom, infer_y0, selected_col_num2, y1_sum)
deci_y2#なし

In [ ]:
# 出力複数、yに含まれる1を固定
def decide_y_fix_y1_mul(X, ori_y, infer_y, selected_col_num, y1_sum):
    select_cols, else_cols = featrure_InnerProduct_sel_mul(X, ori_y, selected_col_num)
    deci_y_list = []
    dot_list = [np.dot(X[i], infer_y) - np.dot(X[j], infer_y) for i in select_cols for j in else_cols]
    
    if len([ll for ll in dot_list if ll <0]) == 0 and sum(infer_y) == y1_sum:
        deci_y_list.append(copy.deepcopy(infer_y))
        
    for kk in reversed(range(num_samples)):
        for k in reversed(range(kk+1)):
            infer_y[k] = 1 - infer_y[k]
            dot_list = [np.dot(X[i], infer_y) - np.dot(X[j], infer_y) for i in select_cols for j in else_cols]
            if len([ll for ll in dot_list if ll <0]) == 0 and sum(infer_y) == y1_sum:
                if (len(deci_y_list) == 0) or (deci_y_list[-1] == infer_y).all() == False:
                    deci_y_list.append(copy.deepcopy(infer_y))
            infer_y[k] = 1 - infer_y[k]
        infer_y[kk] = 1 - infer_y[kk]
        
    return deci_y_list

In [ ]:
infer_y0 = make(num_samples)
deci_y3 = decide_y_fix_y1_mul(X_mushroom_ad, y_mushroom, infer_y0, selected_col_num2, y1_sum)
deci_y3

In [ ]:
check_decide_y_fuc_mul(X_mushroom_ad, y_mushroom, selected_col_num2, deci_y3)

In [ ]:
dot_list_confirm3 = [np.dot(X_mushroom_ad[i], deci_y3[0]) - np.dot(X_mushroom_ad[j], deci_y3[0]) for i in select_cols for j in else_cols]
dot_list_confirm3